# 2.4 그래디언트 기반 최적화

- output = relu(dot(W, input)+b)
    - W와 b는 해당 layer의 속성으로 볼 수 있음
    - 각각 Weight(가중치), bias(편향)이라 하며, trainable parameter(훈련된느 파라미터)라고 함
    - 초기에는 W, b가 무작위로 초기화되며, training하면서 업데이트 됨
    
    
- 훈련 반복 루프(training loop)
    1. 훈련 샘플 x와 타겟 y의 batch를 추출
    2. x를 이용해 네트워크를 실행하고, 예측 y_prediction을 구함
    3. y_prediction, y의 차이를 측정해 손실을 계산
    4. 손실이 감소되도록 네트워크의 모든 가중치를 업데이트
    

- 가중치 업데이트를 어떻게 할 것인가?
    - 신경망에 사용된 모든 연산이 미분 가능(differentiable)하다는 장점을 사용해 네트워크 가중치에 대한 손실의 그래디언트(gradient)를 계산하는 방법을 사용해 가중치를 업데이트

## 2.4.1 변화율

- 연속성
    - 실수 x를 실수 y로 매핑하는 연속적이고 매끄러운 함수 f(x) = y
    - 위 함수는 연속적이므로, x를 조금(epsilon_x) 바꾸면 y도 조금(epsilon_y) 바뀌는데 이것을 연속성이라고 함
    - ∴ f(x+epsilon_x) = y + epsilon_y


- 변화율(derivative)
    - epsilon_x가 아주 작다면, 지점 p에서 기울기 a의 선형 함수로 f를 근사할 수 있음
    - ∴ epsilon_y = a * epsilon_x
    - 이 선형적인 근사는 x가 매우 작아서 p에 매우 가까울 때 유효하며, 이 기울기 a를 p점에서 f의 변화율(derivative)라고 함

## 2.4.2 텐서 연산의 변화율 : 그래디언트

- 그래디언트(gradient)
    - 그래디언트는 텐서 연산의 변화율을 나타냄
    - 텐서를 입력으로 받는 함수에 변화율 개념을 확장시킨 것 
    - 쉽게 말해 2.4.1에서의 f(x)를 f(tensor)로 보고, 이거 미분한다는 얘기
    

- gradient(f)(W)
    - 입력 벡터 x, 행렬 W, 타깃 y, 손실함수 loss라고 할 때, (1)W를 사용해 예측 y_pred를 구하고 (2)예측 y_pred와 타깃 y 사이의 오차를 계산하는 것을 식으로 나타내면 아래와 같음
    - 1 : y_pred = dot(W, x)
    - 2 : loss_value = loss(y_pred, y)
    - 여기서 입력 데이터 x와 y가 고정이라면 1, 2를 아래의 함수(텐서 연산)로 볼 수 있음
    - loss_value = f(W)
    - 따라서, 텐서 연산의 변화율이 gradient이므로, 위 함수(텐서 연산)의 변화율은 다음과 같음
    - gradient(f)(W)
    

- loss가 작아지도록 Weight를 업데이트하는 개념
    - 변화율 개념을 gradient(f)(W)에 적용하면, gradient(f)(W0)는 W0에서의 loss의 변화율(기울기)
    - Ex) W1 = W0 - step * gradient(f)(W0)
    - 위 예의 경우 기울기가 작아지는 곡면의 낮은 위치로 이동한다는 의미
    - ***따라서, gradient의 반대방향(기울기가 작아지는 방향)으로 Weight를 업데이트하다보면 loss가 점점 더 작아지게 됨***
    - 여기서 step이란 gradient(f)(W0)는 W0에 아주 가까이 있을 때의 기울기를 근사한 것이기 때문에 W0에서 너무 크게 벗어나지 않게 하기 위한 스케일링 비율을 뜻함(적당히 조정해야함)

## 2.4.3 확률적 경사 하강법

- gradient(f)(W) = 0
    - 함수의 최솟값은 변화율이 0인 지점 => loss의 최솟 값은 그래디언트가 0인 지점
    - 따라서, gradient(f)(W) = 0을 풀면 됨
    

- gradient를 적용한 훈련 반복 루프(training loop)
    1. 훈련 샘플 x와 타겟 y를 추출
    2. x를 이용해 네트워크를 실행하고, 예측 y_prediction을 구함
    3. y_prediction, y의 차이를 측정해 손실을 계산
    4. 손실 함수(f(W))의 그래디언트(gradient(f)(W))를 계산(역방향으로, backward pass)
    5. gradient의 반대방향으로 parameter(Weight)를 조금씩 이동시키며 loss를 감소시킴 (W -= step * gradient)
    

- SGD(Stochastic Gradient Descent)
    - gradient를 적용해 위에서 설명한 방식이 Stochastic Gradient Descent(확률적 경사 하강법)
    - stochastic(확률적)이라는 말은 무작위(random)하다는 것의 과학적 표현
    - 만약, 위 훈련 반복 루프에서
        - 무작위로 batch를 선택해 학습시키는 방식은 mini-batch SGD(batch size 넣어서 쓰는 일반적인 경우)
        - 각 반복마다 하나의 x와 하나의 y를 뽑아서 반복된다면 그냥 SGD
        - 모든 가용한 데이터를 사용해 반복해서 수행하면 batch SGD
        
      
- optimization method(optimizer)에 따른 여러 SGD
    - 업데이트 할 가중치의 계산 시, 현재 gradient 뿐만 아니라 이전에 업데이트된 weight까지 여러 방식으로 고려하는 SGD들도 있음
    - momentum을 사용한 SGD, Adagrad, RMSProp 등
    - 이러한 SGD의 변종들을 optimization method(최적화 방법) 또는 optimizer(옵티마이저)라고 함
    
    
- momentum(모멘텀)
    - momentum은 현재 기울기 값(현재 가속도)뿐만 아니라 (과거의 가속도로 인한) 현재 속도를 함께 고려하여 업데이트 함
    - 현재 gradient 뿐만 아니라 이전에 업데이트 한 parameter에 기초해 W를 업데이트 하는 방식
    - momentum은 SGD의 2가지 문제점(1. 수렴 속도, 2. local minimum)을 해결해줌

In [ ]:
# 단순 모멘텀 구현 예
past_velocity = 0.
momentum = 0.9 # momentum값은 보통 0.9정도를 많이 사용함
while loss > 0.01:
    w, loss, gradient = get_current_parameters()
    velocity = momentum * past_velocity - learning_rate * gradient
    w = w + velocity
    past_velocity = velocity
    update_parameter

## 2.4.4 변화율 연결 : 역전파 알고리즘(Backpropagation)

- Backpropagation
    - 3개의 텐서 연산 a, b, c와 가중치 행렬 W1, W2, W3로 구성된 네트워크 f를 예로 들면 다음과 같음
    - f(W1, W2, W3) = a(W1, b(W2, c(W3)))
    - 이와 같이 연결된 함수는 미적분의 연쇄 법칙(chain rule)을 통해 유도할 수 있음
    - 연쇄 법칙을 gradient 계산에 적용한 것이 역전파 알고리즘(Backpropagation)
    - Backpropagation은 연쇄 법칙을 적용해 최종 손실 값에서부터 시작해 최초 손실 값까지 진행함